In [1]:
import psycopg2
from creds import local_db_password
from fixture_analysis_config import real_results_sql, local_host, local_name, local_port, local_user
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')
conn = psycopg2.connect(dbname=local_name, user =local_user, host=local_host, password=local_db_password, port =local_port)
cur = conn.cursor()

Home vs Away advantage

In [2]:
# Aggregate result perspective
results_2023 = pd.read_sql(real_results_sql, conn)
results_2023.describe()

,fixture_id,home_goal,away_goal,home_shot,away_shot,home_goalie_save_rate,away_goalie_save_rate,home_passes_attempt,away_passes_attempt,home_pass_accuracy,...,home_tackles,away_tackles,home_shot_blocks,away_shot_blocks,home_interception_rate,away_interception_rate,home_dribble_attempt,away_dribble_attempt,home_dribble_success_rate,away_dribble_success_rate
count,3.800000e+02,380.000000,380.000000,380.000000,380.000000,377.000000,377.000000,380.000000,380.000000,380.000000,...,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000
mean,1.035320e+06,1.728947,1.418421,10.952632,8.852632,0.678933,0.678519,485.773684,454.878947,0.815103,...,17.521053,17.784211,0.414302,0.453177,0.019107,0.018239,17.239474,16.236842,0.513758,0.500007
std,1.655571e+02,1.326691,1.258712,4.575127,3.993975,0.266442,0.233950,151.175591,141.927073,0.074700,...,5.762867,5.509480,0.338168,0.387211,0.008842,0.008217,5.825793,5.380676,0.139059,0.151607
min,1.035037e+06,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,189.000000,164.000000,0.483092,...,4.000000,5.000000,0.000000,0.000000,0.001506,0.002024,4.000000,2.000000,0.125000,0.000000
25%,1.035132e+06,1.000000,0.000000,8.000000,6.000000,0.500000,0.500000,373.750000,349.500000,0.779327,...,13.000000,14.000000,0.181818,0.250000,0.012604,0.012420,13.000000,12.000000,0.428571,0.400000
50%,1.035364e+06,2.000000,1.000000,10.000000,8.000000,0.666667,0.666667,477.000000,439.000000,0.826022,...,17.000000,17.000000,0.333333,0.386752,0.018081,0.017220,17.000000,16.000000,0.518519,0.500000
75%,1.035458e+06,3.000000,2.000000,14.000000,11.000000,0.875000,0.833333,592.250000,542.750000,0.869860,...,21.000000,22.000000,0.571429,0.557292,0.024214,0.022482,20.250000,20.000000,0.611111,0.600000
max,1.035553e+06,6.000000,8.000000,27.000000,22.000000,1.000000,1.000000,1000.000000,895.000000,0.944000,...,39.000000,35.000000,2.500000,4.000000,0.075221,0.059406,35.000000,32.000000,0.900000,0.888889


In [3]:
results_2023.columns

Index(['fixture_id', 'home_goal', 'away_goal', 'home_shot', 'away_shot',
       'home_goalie_save_rate', 'away_goalie_save_rate', 'home_passes_attempt',
       'away_passes_attempt', 'home_pass_accuracy', 'away_pass_accuracy',
       'home_tackles', 'away_tackles', 'home_shot_blocks', 'away_shot_blocks',
       'home_interception_rate', 'away_interception_rate',
       'home_dribble_attempt', 'away_dribble_attempt',
       'home_dribble_success_rate', 'away_dribble_success_rate'],
      dtype='object')

In [4]:
from statsmodels.formula.api import ols

def model_home_vs_away(field = None, source_df = None):
    home_df = source_df[[f'home_{field}']]
    home_df =  home_df.rename(columns={f'home_{field}':field})
    home_df['g'] = 1
    home_df = home_df.dropna(axis=0, how='any')
    away_df = source_df[[f'away_{field}']]
    away_df =  away_df.rename(columns={f'away_{field}':field})
    away_df['g'] = 0
    away_df = away_df.dropna(axis=0, how='any')
    reg_df = pd.concat([home_df, away_df])
    lr = ols(f'{field} ~ g', data=reg_df).fit()
    print(f'\n---Results for {field}------\n')
    print(f'R-Suqared: {lr.rsquared}')
    print(f'Abs Effect of Home adv: {lr.params[1]}')
    print(f'Y_Intercept: {lr.params[0]}')
    print(f'Average {field} for Home: {home_df[field].mean()}')
    print(f'Average {field} for Away: {away_df[field].mean()}')
    print(f'P-value: {1 - lr.pvalues[1]}')

Shooting

In [5]:
model_home_vs_away('goal', results_2023)
model_home_vs_away('shot', results_2023)
model_home_vs_away('goalie_save_rate', results_2023)


---Results for goal------

R-Suqared: 0.014247941190916547
Abs Effect of Home adv: 0.3105263157894746
Y_Intercept: 1.418421052631581
Average goal for Home: 1.7289473684210526
Average goal for Away: 1.418421052631579
P-value: 0.9990228933710367

---Results for shot------

R-Suqared: 0.05655069229760945
Abs Effect of Home adv: 2.100000000000008
Y_Intercept: 8.852631578947367
Average shot for Home: 10.952631578947368
Average shot for Away: 8.852631578947369
P-value: 0.9999999999687377

---Results for goalie_save_rate------

R-Suqared: 6.846577761399075e-07
Abs Effect of Home adv: 0.0004143660642312733
Y_Intercept: 0.678518769765455
Average goalie_save_rate for Home: 0.6789331358296875
Average goalie_save_rate for Away: 0.6785187697654541
P-value: 0.018096900892202727


Passing

In [6]:
model_home_vs_away('passes_attempt', results_2023)
model_home_vs_away('pass_accuracy', results_2023)


---Results for passes_attempt------

R-Suqared: 0.011006145736924933
Abs Effect of Home adv: 30.894736842105633
Y_Intercept: 454.8789473684211
Average passes_attempt for Home: 485.7736842105263
Average passes_attempt for Away: 454.87894736842105
P-value: 0.9962133279481314

---Results for pass_accuracy------

R-Suqared: 0.006348717088332956
Abs Effect of Home adv: 0.012061142096007699
Y_Intercept: 0.8030415913934156
Average pass_accuracy for Home: 0.8151027334894221
Average pass_accuracy for Away: 0.8030415913934151
P-value: 0.9719417835838443


Dribbles

In [7]:
model_home_vs_away('dribble_attempt', results_2023)
model_home_vs_away('dribble_success_rate', results_2023)


---Results for dribble_attempt------

R-Suqared: 0.007949480294531908
Abs Effect of Home adv: 1.0026315789473903
Y_Intercept: 16.236842105263158
Average dribble_attempt for Home: 17.239473684210527
Average dribble_attempt for Away: 16.236842105263158
P-value: 0.9860605664649023

---Results for dribble_success_rate------

R-Suqared: 0.002234898148463671
Abs Effect of Home adv: 0.013751223181149241
Y_Intercept: 0.5000065423317217
Average dribble_success_rate for Home: 0.5137577655128701
Average dribble_success_rate for Away: 0.5000065423317216
P-value: 0.807034577294297


Defense

In [8]:
model_home_vs_away('tackles', results_2023)
model_home_vs_away('shot_blocks', results_2023)
model_home_vs_away('interception_rate', results_2023)


---Results for tackles------

R-Suqared: 0.0005458734586717418
Abs Effect of Home adv: -0.26315789473681567
Y_Intercept: 17.784210526315796
Average tackles for Home: 17.521052631578947
Average tackles for Away: 17.78421052631579
P-value: 0.47985820774442756

---Results for shot_blocks------

R-Suqared: 0.0028583667540043045
Abs Effect of Home adv: -0.03887438138496355
Y_Intercept: 0.45317650558198813
Average shot_blocks for Home: 0.41430212419702395
Average shot_blocks for Away: 0.45317650558198797
P-value: 0.8591193866826918

---Results for interception_rate------

R-Suqared: 0.002582862803528707
Abs Effect of Home adv: 0.0008674893689284734
Y_Intercept: 0.01823936012242494
Average interception_rate for Home: 0.01910684949135339
Average interception_rate for Away: 0.018239360122424934
P-value: 0.8383850815293552
